In [1]:
%pip install groq # Instalando o groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 3.4 MB/s eta 0:00:00


In [2]:
import os

os.environ['GROQ_API_KEY'] = "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" # Utilizando a chave da API que peguei no Groq

In [3]:
import os
from groq import Groq # Importa a classe que nos permite interagir com a API do Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
) # Inicializando o Groq e passando a minha chave ali de cima

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models", # Passando as mensagens para o nosso modelo
        }
    ],
    model="llama3-70b-8192", # Definindo que modelo vamos utilizar
)

print(chat_completion.choices[0].message.content) # Aqui printamos a resposta que o modelo vai dar para o nosso prompt/mensagem que passamos para ele ali em cima

Fast language models are crucial in today's natural language processing (NLP) landscape, and their importance can be seen in several aspects:

1. **Real-time Processing**: Fast language models enable real-time processing of large volumes of text data, which is essential for applications like chatbots, virtual assistants, and sentiment analysis. They can quickly analyze and respond to user input, providing a more interactive and engaging experience.
2. **Efficient Deployment**: Fast language models can be deployed on devices with limited computational resources, such as smartphones, embedded systems, or edge devices. This allows for on-device NLP capabilities, reducing latency, and improving user privacy by avoiding the need to send data to cloud servers for processing.
3. **Scalability**: Fast language models can handle large datasets and high traffic volumes, making them ideal for applications that require processing massive amounts of text data, such as search engines, social media p

In [4]:
class Agent:
  def __init__(self, client, system):
    self.client =  client # Inicializando o cliente
    self.system = system # Inicializando a mensagem do sistema para o modelo entender como ele deve trabalhar em loop
    self.messages = [] # Lista de mensagens que vamos passar
    if self.system is not None: # Se a mensagem não for vazia nos vamos adiciona-la na nossa lista
      self.messages.append({"role":"system", "content": self.system})

  def __call__(self, message=""):
      if message: # Vai ser ativado sempre que o nosso agente for chamado
        self.messages.append({"role":"user", "content": message}) # Se receber uma mensagem vai adiciona-la na lista
      result = self.execute() # Depois de receber a mensagem vamos executar o agente com todas as mensagens da nossa lista
      self.messages.append({"role":"assistant", "content": result}) # Depois vai adicionar o resultado na nossa lista
      return result # E por fim retornar o resultado para gente

  def execute(self): # Método de execução que usamos no método anterior após recebermos a mensagem
      completion = client.chat.completions.create(
          messages= self.messages ,
          model="llama3-70b-8192", # Definindo que modelo vamos utilizar
      )
      return completion.choices[0].message.content

In [5]:
system_prompt  = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_population:
e.g. get_population: Goiania
returns population of Goiania

Example session:

Question: What is the population of Goiânia times 2?
Thought: I need to find the population of Goiania
Action: get_population: Goiania
PAUSE

You will be called again with this:

Observation: 1555626

Thought: I need to multiply this by 2
Action: calculate: 1555626 * 2
PAUSE

You will be called again with this:

Observation: 3111252

If you have the answer, output it as the Answer.

Answer: The population of Goiânia times 2 is 3111252

Now it's your turn:
""".strip()

# Esse o prompt do sistema que vai fazer o agente entender como ele deve se comportar para seguir o loop

In [7]:
# Ferramentas
def calculate(operation):
  return eval(operation) # Ferramenta padrão de calculadora

def get_population(city) -> float: # Ferramenta que serve como uma "pseudo-pesquisa" para que o agente possa realizar o que lhe foi pedido
  match city.lower():
    case "goiania":
      return 1555626
    case "sao paulo":
      return 11895578
    case "rio de janeiro":
      return 6211223
    case "brasilia":
      return 2982818

In [8]:
ibge = Agent(client, system_prompt) # Instanciando o agente com o nome IBGE porque ele é um especialista em populações de cidades

**TESTANDO O AGENTE MANUALMENTE**

In [9]:
result = ibge("What is the sum of the population of Sao Paulo and Rio de Janeiro times 2?")
print(result)

Thought: I need to find the population of Sao Paulo and Rio de Janeiro.


In [10]:
ibge.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_population:\ne.g. get_population: Goiania\nreturns population of Goiania\n\nExample session:\n\nQuestion: What is the population of Goiânia times 2?\nThought: I need to find the population of Goiania\nAction: get_population: Goiania\nPAUSE\n\nYou will be called again with this:\n\nObservation: 1555626\n\nThought: I need to multiply this by 2\nAction: calculate: 1555626 * 2\nPAUSE\n\nYou will be called again with this:\n\nObservation: 3111252\n\nIf you have 

In [12]:
result = ibge()
print(result)

Action: get_population: Sao Paulo
PAUSE


In [13]:
observation = get_population("sao paulo")
print (observation)

11895578


In [14]:
next_prompt = f"Observation: {observation}"
result = ibge(next_prompt)
print(result)

Thought: Now I have the population of Sao Paulo, next I need to find the population of Rio de Janeiro.


In [15]:
ibge.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_population:\ne.g. get_population: Goiania\nreturns population of Goiania\n\nExample session:\n\nQuestion: What is the population of Goiânia times 2?\nThought: I need to find the population of Goiania\nAction: get_population: Goiania\nPAUSE\n\nYou will be called again with this:\n\nObservation: 1555626\n\nThought: I need to multiply this by 2\nAction: calculate: 1555626 * 2\nPAUSE\n\nYou will be called again with this:\n\nObservation: 3111252\n\nIf you have 

In [17]:
result = ibge()
print(result)

Action: get_population: Rio de Janeiro
PAUSE


In [18]:
observation = get_population("rio de janeiro")
print (observation)

6211223


In [19]:
next_prompt = f"Observation: {observation}"
result = ibge(next_prompt)
print(result)

Thought: Now I have the population of Rio de Janeiro, next I need to add them together and multiply by 2.


In [20]:
ibge.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_population:\ne.g. get_population: Goiania\nreturns population of Goiania\n\nExample session:\n\nQuestion: What is the population of Goiânia times 2?\nThought: I need to find the population of Goiania\nAction: get_population: Goiania\nPAUSE\n\nYou will be called again with this:\n\nObservation: 1555626\n\nThought: I need to multiply this by 2\nAction: calculate: 1555626 * 2\nPAUSE\n\nYou will be called again with this:\n\nObservation: 3111252\n\nIf you have 

In [22]:
result = ibge()
print(result)

Action: calculate: 11895578 + 6211223
PAUSE


In [23]:
observation = calculate("(11895578 + 6211223)")
print(observation)

18106801


In [24]:
next_prompt = f"Observation: {observation}"
result = ibge(next_prompt)
print(result)

Thought: Now I have the sum of the populations, next I need to multiply it by 2.


In [26]:
result = ibge()
print(result)

Action: calculate: 18106801 * 2
PAUSE


In [27]:
observation = calculate("18106801 * 2")
print(observation)

36213602


In [28]:
next_prompt = f"Observation: {observation}"
result = ibge(next_prompt)
print(result)

Answer: The sum of the population of Sao Paulo and Rio de Janeiro times 2 is 36213602


In [29]:
ibge.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_population:\ne.g. get_population: Goiania\nreturns population of Goiania\n\nExample session:\n\nQuestion: What is the population of Goiânia times 2?\nThought: I need to find the population of Goiania\nAction: get_population: Goiania\nPAUSE\n\nYou will be called again with this:\n\nObservation: 1555626\n\nThought: I need to multiply this by 2\nAction: calculate: 1555626 * 2\nPAUSE\n\nYou will be called again with this:\n\nObservation: 3111252\n\nIf you have 

**CRIANDO O LOOP PARA O AGENTE FUNCIONAR APENAS COM O PROMPT**

In [30]:
import re # Criando o loop do agente

def agent_loop(max_iterations, system, query):
  agent = Agent(client, system_prompt) # Inicilizando o agente
  tools = ['calculate', 'get_population'] # Inicilizando as ferramentas que temos
  next_prompt = query # Definimos o primeiro prompt como pedido que vamos mandar para o loop do agente
  i = 0
  while i < max_iterations: # Iniciando o loop
    i += 1
    result = agent(next_prompt) # Chamando o agente
    print(result) # Printando o resultado para análise

    if "PAUSE" in result and "Action" in result: # Se o resultado tiver PAUSE e AÇÃO
      action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE) # Nós encontramos o nome da ação e os parâmetros com essa linha de código
      chosen_tool = action[0][0] # Organizando a ferramenta selecionada
      arg = action[0][1] # Organizado o parâmetro encontrado

      if chosen_tool in tools: # Se a ferramenta estiver entre as que nós temos
        result_tool = eval(f"{chosen_tool}('{arg}')") # Usamos e ferramenta e temos o resultado dela
        next_prompt = f"Observation: {result_tool}" # Que é analisado nessa observação

      else:
        next_prompt = "Observation: Tool not found" # Se não tivermos a ferramenta vamos dizer que não encontramos a ferramenta para que o sistema procure outra correta

      print(next_prompt) # Printamos para sabermos o que está acontecendo
      continue # Continua a fazer mais iterações

    if "Answer" in result: # Se o agente tiver "resposta" no seu resultado nós paramos o loop
      break


In [31]:
agent_loop(max_iterations=50, system=system_prompt, query="What is the sum of the population of Sao Paulo, Goiania and Rio de Janeiro and all of it times 5?") # Implementando o loop do agente criado

Thought: I need to find the population of Sao Paulo, Goiania, and Rio de Janeiro, then add them up and finally multiply the sum by 5.

Action: get_population: Sao Paulo
PAUSE
Observation: 11895578
Thought: I have the population of Sao Paulo, now I need to get the populations of Goiania and Rio de Janeiro.

Action: get_population: Goiania
PAUSE
Observation: 1555626
Thought: I have the populations of Sao Paulo and Goiania, now I need to get the population of Rio de Janeiro.

Action: get_population: Rio de Janeiro
PAUSE
Observation: 6211223
Thought: I have the populations of Sao Paulo, Goiania, and Rio de Janeiro, now I need to add them up.

Action: calculate: 11895578 + 1555626 + 6211223
PAUSE
Observation: 19662427
Thought: I have the sum of the populations, now I need to multiply it by 5.

Action: calculate: 19662427 * 5
PAUSE
Observation: 98312135
Thought: I have the result of multiplying the sum by 5, which is the final answer.

Answer: The sum of the population of Sao Paulo, Goiania,